In [5]:
!pip install -U transformers
!pip install -U torch

  Using cached torch-2.8.0-cp311-cp311-win_amd64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached torch-2.8.0-cp311-cp311-win_amd64.whl (241.4 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl (15 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 14.2 MB/s  0:00:00

   ---------------------------------------- 0/6 [mpmath]
   ---------------------------------------- 0/6 [mpmath]
   ---------------------------------------- 0/6 [

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

d:\Downloads\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
context = 'Theo pháp lệnh Vincennes năm 1374, vương quốc được điều hành bởi Nhiếp chính vương cho đến khi Louis lên 13 tuổi. Danh hiệu Nhiếp chính được trao cho người bà con gần nhất là ông chú của nhà vua Philippe, Quận công xứ Orleans. Louis XIV, tuy nhiên, không tín nhiệm Philippe, một người lính kiệt xuất, nhưng bị nhà vua coi là kẻ không sùng đạo. Nhà vua gọi Philippe là Fanfaron des crimes ("đầu sỏ của tội ác)" Louis XIV muốn quyền điều hành Hội đồng Nhiếp chính phải giao cho người con ngoại hôn được ông rất thương yêu, Quận công xứ Maine (con triêng của Louis XIV với Madame de Montespan). Tháng 8 năm 1714, không lâu trước khi chết, nhà vua viết di chiếu lệnh hạn chế quyền hạn của người chấp chính; theo đó quốc gia sẽ được điều hành bởi Hội đồng Nhiếp chính gồm 14 thành viên cho đến năm tân vương 13 tuổi. Philippe là cháu gọi Louis XIV là bác, làm Chủ tịch Hội đồng, nhưng còn có các thành viên khác bao gồm Quận công xứ Maine cùng các đồng minh. Quyết định của triều đình được ban xuống theo chế độ đa số phiếu, nghĩa là quyền lực Nhiếp chính vương có thể bị bác bỏ bởi nhóm Maine. Orléans nhìn ra điều đó, và ngay sau khi nhà vua qua đời, ông đến Nghị viện Paris, một Hội đồng quý tộc gồm nhiều đồng minh của ông, và Nghị viện đã hủy bỏ tờ di chiếu. Để đổi lấy sự ủng hộ của họ, Orléans cho khôi phục droit de remontrance (quyền phản đối) của Nghị viện - vốn bị Louis XIV triệt bỏ từ trước, theo đó Nghị viện có quyền phản đối những quyết định của nhà vua mà họ cho là trái với lợi ích dân tộc. Quyền phản đối làm suy yếu quyền hành của quân chủ và đánh dấu khởi đầu xung đột giữa Nhà vua và Nghị viện mà đỉnh điểm là Cách mạng Pháp năm 1789.'
question = 'Quyền phản đối được khôi phục bởi Orléans đã giúp cho Nghị viện có được quyền lực nào, mặc dù nhà vua luôn duy trì quyền lực tuyệt đối và không bao giờ cho phép bất kỳ sự can thiệp nào từ Nghị viện?'

In [5]:
messages = [
    {
        "role": "system",
        "content": "Bạn là một trợ lý hữu ích. Trả lời câu hỏi của người dùng bằng tiếng Việt, chỉ dựa trên thông tin trong phần Context được cung cấp."
    },
    {
        "role": "user",
        "content": f"Ngữ cảnh: {context}\nCâu hỏi: {question}\nHãy trả lời ngắn gọn trong một đoạn văn."
    }
]


# Tạo input từ template chat
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

# Sinh output
outputs = model.generate(
    **inputs,
    max_new_tokens=2042,        # giới hạn token sinh
    do_sample=True,            # cho random sampling (nếu muốn sáng tạo hơn)
    top_p=0.9,                 # nucleus sampling
    temperature=0.7            # kiểm soát độ đa dạng
)

# Lấy phần text mới sinh
generated_answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[-1]:],
    skip_special_tokens=True   # bỏ token đặc biệt
)

print("Generated Answer:", generated_answer)


Generated Answer: Quyền phản đối được khôi phục bởi Orléans đã giúp Nghị viện có được quyền lực kiểm soát quan trọng hơn về quyết định của nhà vua, mặc dù nhà vua luôn duy trì quyền lực tuyệt đối và không bao giờ cho phép bất kỳ sự can thiệp nào từ Nghị viện.


In [7]:
generated_answer = 'Quyền phản đối được khôi phục bởi Orléans đã giúp Nghị viện có được quyền lực kiểm soát quan trọng hơn về quyết định của nhà vua, mặc dù nhà vua luôn duy trì quyền lực tuyệt đối và không bao giờ cho phép bất kỳ sự can thiệp nào từ Nghị viện.'

In [2]:
!git clone https://github.com/thu-coai/CTRLEval.git
%cd CTRLEval
!pip install torch transformers numpy nltk scipy tqdm

d:\Downloads\CTRLEval


fatal: destination path 'CTRLEval' already exists and is not an empty directory.


In [3]:
from ctrleval import CTRLEval

# ví dụ với task sentiment
task = "senti"   # hoặc "topic"

scorer = CTRLEval(
    iwf_dir="iwf_full.txt",
    prompt_dir=f"./prompt/prompt_{task}.txt",
    verbal_dir=f"./prompt/verbal_{task}.txt",
    model_name_or_path="google/pegasus-large",
    device="cpu"
)

d:\Downloads\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import re
generated_answer = re.sub(r'\*+', '', generated_answer)   # bỏ tất cả dấu *

In [9]:
question

'Quyền phản đối được khôi phục bởi Orléans đã giúp cho Nghị viện có được quyền lực nào, mặc dù nhà vua luôn duy trì quyền lực tuyệt đối và không bao giờ cho phép bất kỳ sự can thiệp nào từ Nghị viện?'

In [10]:
prefix = [question]   # context = câu hỏi
data = [question + '\n' + generated_answer]  # context + câu trả lời

cons = scorer.score(aspect="cons", data=data, prefix=prefix)      # consistency

print("Consistency:", cons)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:01<00:00,  1.65it/s]

Consistency: [np.float64(-0.575809121131897)]


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("dangvantuan/vietnamese-document-embedding", trust_remote_code=True)

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

torch.Size([3, 3])


In [ ]:
embeddings.shape

(3, 768)